This a google Colab notebook based on Data camp Deep Seek fine tuning tutorial https://www.datacamp.com/tutorial/fine-tuning-deepseek-r1-reasoning-model which I have adapted to for translating standard Finnish to South Osthrobothnian dialect. Fine tuning data is done in two passes firstly a bit over 300 standard Finnish and South Osthrobothnian dialect sentence pairs and second on bit over 2000 standard Finnish South Osthrobothnia word pairs. South Osthrobothnian sentences are taken from Pekka Pohjanpää's blog: http://pohopekka.blogspot.com/" I'm not sharing the sentence pairs but Standard Finnish that correspond the blog text. Blog text is indexed by sentence order so that standard Finnish sentences can be matched. I also share the vocabulary. Shared data is placeded at directory data.
Not that the blog is Copyrighted material and I haven't reached copyright holder. Therfore I provide only a download function. Use at your own responsibility.

In [ ]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git


In [ ]:
!pip install bs4

Directory settings. Adapt the as needed. Secrets are needed for huggingface and wandb tokens. Datadir is needed for storing General Finnish - South Ostrobotnian vocabulary data in file vocabulary_deep_seek_format.json and dialect sentence indices against standard Finnish sentences file in standard_finnish_sentences.json. I'm using google colab, and therefore directories point to google drive.

In [ ]:
SECRETS_DIR = "/content/drive/My Drive/secrets"
DATA_DIR = "/content/drive/My Drive/data"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

secret_file_path = SECRETS_DIR+'/huggingfacetoken.key'
with open(secret_file_path) as f:
  hf_token=f.read().strip()

In [ ]:
secret_file_path = SECRETS_DIR+'/wandb.key'

with open(secret_file_path) as f:
  wb_token=f.read().strip()

In [ ]:
from huggingface_hub import login
login(hf_token)

Wandb is used here to monitor training. You'll need to create account and get token to use it.

In [ ]:
import wandb
import os
os.environ["WANDB_SILENT"] = "true"

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on South Ostrobotnia dialect 2',
    job_type="training",
    anonymous="allow"
)

Deep Seek model is downloaded from Huggingface. You'll need Huggingface api key for this.

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token,
)

Prompt for testing with standard Finnish how that translates to the dialect.

In [ ]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
Translate standard Finnish sentence to South Ostrobothnian dialect.

### Question:
{}

### Response:
<think>{}"""

Run test on the original model before finetuning.

In [ ]:
standard_finnish = "Hevoskauppias ajoi uudella hevosellaan meidän talon ohi. Katsoin häntä aitan ovelta."

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(standard_finnish, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print("Dialect:")
print(response[0].split("### Response:")[1])

Training prompt for standard Finnish and dialect sentence pairs.

In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
Translate standard Finnish sentence to South Ostrobothnian dialect.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

Training prompt for standard Finnish to dialect vocabulary.

In [ ]:
train_prompt_vocabulary_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
Translate the following word from standard Finnish to South Ostrobothnian dialect.

### Word:
{}

### Dialect Equivalent:
<think>
{}
</think>
{}"""

Formatting function for standard Finnish to dialect sentence pairs.

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

Formatting function for vocabulary prompts.

In [ ]:
def formatting_prompts_vocab_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_vocabulary_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

Function to get Pekka Pohjanpää's blog. The blog is split into sentence level and arranged as list of dictionaries.

In [ ]:
import random
import requests
from bs4 import BeautifulSoup
import json
def get_pohopekka():
    main_page_link="http://pohopekka.blogspot.com/"
    structured_data = []
    page=requests.get(main_page_link)
    soup = BeautifulSoup(page.content, 'html.parser')
    for element in soup.find_all(["h2", "div"]):
        if element.name=="h2":
            latest_h2 = element.text.strip()
        elif element.name=="div" and "blogPost" in element.get("class", []):
            if latest_h2:
                content=element.text.split("# posted")[0].strip().replace('\r',' ').replace('PEKAN SIVUPERSOONA','').\
                    replace('Pekan Sivupersoona','').replace("\n"," ")

                content=content.split("----",1)[0].strip()

                if len(content)>5:
                    structured_data.append({"title": latest_h2, "content": content})
                latest_h2 = None

    return structured_data

Matching Pekka Pohjanpää's story on dialect to standard Finnish based on precalculated index.

In [ ]:
def match_story_sentences_to_standard(stories,standard_finnish_sentences):
    story_idx=0
    dialect=dict()
    for story_ in stories:
        story=story_["content"]
        story_splitted=story.split(". ")
        inside_story_idx=0
        for sentence in story_splitted:
            dialect[str(story_idx) + "-" + str(inside_story_idx)] = sentence
            inside_story_idx+=1
        story_idx+=1


    with open(standard_finnish_sentences) as f:
        standard=json.load(f)

    standard_dialect=[]

    for key in standard:
        standard_dialect.append({"standard":standard[key],"dialect":dialect[key]})
    return standard_dialect

Function to compose standard dialect prompts.

In [ ]:
def write_translated(standard_dialect):
    data_arr=[]
    for data in standard_dialect:
        target_data=data["dialect"].strip()
        input_data=data["standard"].strip()
        system_dict=dict()
        input_dict=dict()
        target_dict=dict()
        system_dict["role"] = "system"
        system_dict["content"]="You translate standard Finnish sentences into the South Ostrobothnian dialect."
        input_dict["role"] = "user"
        input_dict["content"] = "Ilmaise seuraava Etelä-Pohjanmaan murteella: " + input_data

        target_dict["role"] = "assistant"
        target_dict["content"] = target_data
        data_arr.append({
            "Question": input_data,  # Standard Finnish
            "Complex_CoT": "",
            "Response": target_data,  # South Ostrobothnian dialect
            "text": input_data + " - " + target_data  # Same as Question and Response
        })


    return data_arr

In [ ]:
stories=get_pohopekka()
standard_dialect=match_story_sentences_to_standard(stories=stories,standard_finnish_sentences=DATA_DIR+"/standard_finnish_sentences.json")
train_set=write_translated(standard_dialect=standard_dialect)
random.shuffle(train_set)

Run standard Finnish vs. dialect data generation.

In [ ]:
from datasets import load_dataset, Dataset

dataset0 = Dataset.from_list(train_set)

dataset0 = dataset0.map(formatting_prompts_func, batched = True,)

Get the vocabulary data, which I'm sharing in the repo.

In [ ]:
data_file_path = DATA_DIR+'/vocabulary_deep_seek_format.json'


dataset1 = load_dataset('json',data_files=data_file_path, split = "train[:-1]",trust_remote_code=True)

dataset1 = dataset1.map(formatting_prompts_vocab_func, batched = True,)

Model configurations.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Trainer for standard Finnish - dialect sentence pairs.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer_transl = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset0,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        #max_steps=60,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Trainer for vocabulary data. Less epochs and smaller learning rate as number of training samples is larger than number of samples on the sentence pair data.

In [ ]:
#works when not ran
trainer_vocab = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset1,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        #max_steps=60,
        num_train_epochs=1,
        learning_rate=5e-5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs/vocab",
    ),
)

Run trainers.

In [ ]:
!nvidia-smi
trainer_stats = trainer_transl.train()

In [ ]:
!nvidia-smi
trainer_stats = trainer_vocab.train()

Run test on single sentence.

In [ ]:
standard_finnish = "Hevoskauppias ajoi uudella hevosellaan meidän talon ohi. Katsoin häntä aitan ovelta."


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(standard_finnish, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print("Dialect:")
print(response[0].split("### Response:")[1])


Save and produce a quantized smaller sized model < 10 Gb

In [ ]:
import shutil
new_model_local = "DeepSeek-R1-South-Ostrobothnia"
model.save_pretrained(new_model_local)
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

In [ ]:
!ls -l ./DeepSeek-R1-South-Ostrobothnia
shutil.make_archive("deepseek_model", 'zip', new_model_local)

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp




In [ ]:
os.chdir("llama.cpp")
!ls
!pip install -r ./requirements.txt

In [ ]:
import os
os.chdir("llama.cpp")

!python3 ./convert_hf_to_gguf.py ../DeepSeek-R1-South-Ostrobothnia --outfile deepseek_south_ostrobothnia.gguf


In [ ]:
#os.makedirs("build",exist_ok=True)
os.chdir("build")
!cmake ..
!cmake --build . --config Release

In [ ]:
os.chdir("llama.cpp/")

In [ ]:
!build/bin/llama-quantize deepseek_south_ostrobothnia.gguf deepseek_south_ostrobothnia.Q5_K_M.gguf Q5_K_M


In [ ]:

!ls
!cp deepseek_south_ostrobothnia.Q5_K_M.gguf "/content/drive/My Drive/models"
